In [17]:
import pandas as pd

In [18]:
df = pd.read_csv('datascience.csv',encoding="gb18030")

In [19]:
df.head()

title    author  \
0                   大数据产业迎政策暖风 最新大数据概念股一览    财经热点扒客   
1  Google发布机器学习平台Tensorflow游乐场～带你一起玩神经网络！      硅谷周边   
2                     李克强：中国大数据和云计算产业是开放的  苏州高新区金融办   
3                             全峰集团持续挖掘大数据     快递物流网   
4         第366期【微理工】贵州理工学院召开大数据分析与应用专题分享会    贵州理工学院   

                                             content  
0  大数据产业发展受到国家重视，而大数据已经上升为国家战略，未来发展前景很广阔。大数据产业“十三...  
1  点击上方“硅谷周边”关注我，收到最新的文章哦！昨天，Google发布了Tensorflow游...  
2  国务院总理李克强当地时间20日上午在纽约下榻饭店同美国经济、金融、智库、媒体等各界人士座谈，...  
3  2016年，全峰集团持续挖掘大数据、云计算、“互联网+”等前沿技术和物流快递的融合，并通过优...  
4  贵州理工学院召开大数据分析与应用专题分享会 借“创响中国”贵安站巡回接力活动暨2016贵安大...

In [20]:
import jieba
def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))

In [21]:
df['content_cutted'] = df.content.apply(chinese_word_cut)

In [22]:
df.content_cutted.head()

0    大 数据 产业 发展 受到 国家 重视 ， 而 大 数据 已经 上升 为 国家 战略 ， 未...
1    点击 上方 “ 硅谷 周边 ” 关注 我 ， 收到 最新 的 文章 哦 ！ 昨天 ， Goo...
2    国务院 总理 李克强 当地 时间 20 日 上午 在 纽约 下榻 饭店 同 美国 经济 、 ...
3    2016 年 ， 全峰 集团 持续 挖掘 大 数据 、 云 计算 、 “ 互联网 + ” 等...
4    贵州 理工学院 召开 大 数据分析 与 应用 专题 分享 会   借 “ 创响 中国 ” 贵...
Name: content_cutted, dtype: object

In [23]:
#向量化处理
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [24]:
n_features = 1000
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(df.content_cutted)


In [25]:
from sklearn.decomposition import LatentDirichletAllocation

In [26]:
n_topics = 5
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)


In [27]:
lda.fit(tf)

c:\program files\anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=10, n_jobs=1, n_topics=5, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

主题没有一个确定的名称，而是用一系列关键词刻画的。我们定义以下的函数，把每个主题里面的前若干个关键词显示出来

In [30]:
def print_top_words(model,feature_names,n_top_words):
    for topic_idx,topic in enumerate(model.components_):
        print("Topic #%d:"%topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
        


In [31]:
n_top_words = 20
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
学习 模型 使用 算法 方法 机器 可视化 神经网络 特征 处理 不同 计算 用户 数据库 系统 如果 分类 训练 一种 基于
Topic #1:
这个 就是 可能 没有 如果 他们 自己 很多 什么 不是 但是 或者 因为 时候 这样 现在 电子 一些 所以 孩子
Topic #2:
企业 平台 服务 管理 互联网 数据分析 公司 产品 用户 业务 行业 客户 金融 创新 实现 价值 系统 能力 工作 需求
Topic #3:
中国 2016 市场 增长 10 城市 用户 2015 关注 行业 其中 30 人口 检索 阅读 大众 投资 全国 美国 20
Topic #4:
人工智能 学习 领域 智能 机器人 机器 人类 公司 深度 研究 未来 识别 已经 系统 计算机 目前 医疗 语音 方面 服务


In [34]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
data = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)
pyLDAvis.show(data)

c:\program files\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  topic_term_dists = topic_term_dists.ix[topic_order]



Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]


127.0.0.1 - - [28/Mar/2018 17:34:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Mar/2018 17:34:24] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [28/Mar/2018 17:34:24] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Mar/2018 17:34:24] "GET /LDAvis.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Mar/2018 17:34:24] code 404, message Not Found
127.0.0.1 - - [28/Mar/2018 17:34:24] "GET /favicon.ico HTTP/1.1" 404 -



stopping Server...


c:\program files\anaconda3\lib\site-packages\simplejson\encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2018-03-28 17:34:12.610129. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)
